# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "Humans" & df$list_id != -1, ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
df$Side <- df$Angle != "front"

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156825 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20.0.0 (64-bit)
Running under: macOS 15.5

Matrix products: default
BLAS/LAPACK: /Users/zory/miniforge3/envs/py310/lib/libopenblas.0.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] car_3.1-3              carData_3.0-5          emmeans_1.11.0        
[4] marginaleffects_0.25.1 readr_2.1.5            lme4_1.1-35.5         
[7] Matrix_1.6-5          

loaded via a namespace (and not attached):
 [1] bit_4.6.0          jsonlite_2.0.0     compiler_4.3.3     crayon_1.5.3      
 [5] tidyselect_1.2.1   Rcpp_1.0.14        IRdisplay_1.1      parallel_4.3.3    
 [9] splines_4.3.3      boot_1.3-31        uuid_1.2-1         fastmap_1.2.0     
[13] IRkernel_1.3.2     lattice_0.22-6     R6_2.6.1           generics_0.1.4    
[17] Formula_1.2-5      MASS_7.3-60

# Statistics

In [3]:
mean(df[ (df$Proximity == 1) & (df$n_candidates == 2), ]$Accuracy)

[1] 0.9929701

In [4]:
length(unique(df$Participant_ID))

[1] 65

# Fitting models

In [5]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.241842 (tol = 0.002, component 1)"


In [6]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
1312.7752 1360.6236 -648.3876 1296.7752      2917 
Random effects:
 Groups         Name        Std.Dev.
 Stimulus_ID    (Intercept) 1.6566  
 Participant_ID (Intercept) 0.6354  
Number of obs: 2925, groups:  Stimulus_ID, 900; Participant_ID, 65
Fixed Effects:
                       (Intercept)                           Angleleft  
                             7.414                              -3.236  
                        Angleright     scale(Proximity, scale = FALSE)  
                            -3.218                              -1.305  
scale(n_candidates, scale = FALSE)                              ActorY  
                  

In [7]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [8]:
# delete Actor and turn Angle into Side
model3 <- glmer(Accuracy ~ Side *  scale(Proximity, scale=FALSE) +  Side *  scale(n_candidates, scale=FALSE) + (1 | Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [9]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,7,1326.876,1368.743,-656.4379,1312.876,NA,NA,NA
model1,8,1312.775,1360.624,-648.3876,1296.775,16.10054,1,6.006683e-05


In [10]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,1312.775,1360.624,-648.3876,1296.775,NA,NA,NA
model3,8,1328.318,1376.167,-656.1591,1312.318,0,0,NA


In [11]:
model <- model1

# Interpretation

In [12]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  1312.8   1360.6   -648.4   1296.8     2917 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.6893  0.0358  0.0892  0.1932  2.0510 

Random effects:
 Groups         Name        Variance Std.Dev.
 Stimulus_ID    (Intercept) 2.7443   1.6566  
 Participant_ID (Intercept) 0.4037   0.6354  
Number of obs: 2925, groups:  Stimulus_ID, 900; Participant_ID, 65

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          7.4138     0.5270  14.068  < 2e-16 ***
Angleleft                           -3.2358     0.4217  -7.673 1.68e-14 ***
Angleright                       

In [13]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$`Pr(>|z|)` <- format(res$`Pr(>|z|)`, scientific = FALSE)
res$CI_low <- res$Estimate - 1.96 * res$`Std. Error`
res$CI_high <- res$Estimate + 1.96 * res$`Std. Error`
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),CI_low,CI_high,effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),7.4138498,0.5270114,14.067722,0.000000000000000000000000000000000000000000005996996,6.3809075,8.4467922,2.537307,2.9219368
Angleleft,-3.2358288,0.4217099,-7.673115,0.000000000000016786913754493492702827715485488177057,-4.0623803,-2.4092773,2.537307,-1.2753006
Angleright,-3.2180958,0.4238400,-7.592713,0.000000000000031327473973537336995396071273437968008,-4.0488222,-2.3873694,2.537307,-1.2683117
"scale(Proximity, scale = FALSE)",-1.3047983,0.1677296,-7.779178,0.000000000000007299743423945285812410353771901962463,-1.6335483,-0.9760483,2.537307,-0.5142454
"scale(n_candidates, scale = FALSE)",-0.6700431,0.1518226,-4.413329,0.000010179310577112439211550608264555961568476050161,-0.9676154,-0.3724708,2.537307,-0.2640765
ActorY,-1.0032615,0.2383074,-4.209947,0.000025543060988475885779045923551500152370863361284,-1.4703439,-0.5361790,2.537307,-0.3954041


In [14]:
avg_predictions(model, by="Angle")

Angle,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
front,0.9958731,0.001661877,599.24588,0,Inf,0.9926159,0.9991304
left,0.9163054,0.014048631,65.22382,0,Inf,0.8887706,0.9438402
right,0.9215027,0.013206891,69.77438,0,Inf,0.8956177,0.9473877


In [15]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.079567747,0.01356133,-5.8672512,4.430791e-09,27.7497885,-0.10614747,-0.05298802
(right) - (front),-0.074370462,0.01272584,-5.8440532,5.094575e-09,27.5483911,-0.09931264,-0.04942828
(right) - (left),0.005197285,0.01418944,0.3662782,7.141575e-01,0.4856859,-0.02261352,0.03300809


In [16]:
avg_predictions(model, by="Proximity")

Proximity,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.9845313,0.004086799,240.90525,0,Inf,0.9765214,0.9925413
2,0.9582795,0.006858015,139.73132,0,Inf,0.9448380,0.9717209
3,0.8942306,0.015564925,57.45165,0,Inf,0.8637240,0.9247373


In [17]:
avg_slopes(model, variables ="Proximity")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.05154791,0.007989378,-6.452055,1.103433e-10,33.07728,-0.0672068,-0.03588902,0.9992916,0.9992914,0.9992915


In [18]:
avg_predictions(model, by="n_candidates")

n_candidates,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.9621340,0.007425048,129.57950,0,Inf,0.9475811,0.9766868
3,0.9352091,0.009146027,102.25305,0,Inf,0.9172832,0.9531350
4,0.9004136,0.015686028,57.40227,0,Inf,0.8696695,0.9311576


In [19]:
avg_slopes(model, variables ="n_candidates")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.026471,0.005828612,-4.541562,5.583886e-06,17.4503,-0.03789487,-0.01504713,0.9992916,0.9992915,0.9992915


In [20]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.04517929,0.01032169,-4.377123,1.202562e-05,16.34353,-0.06540943,-0.02494916


The below code is for generating the table in the paper.

In [21]:
stim_id_var_val <- if (!is.null(vc_list$Stimulus_ID) && length(vc_list$Stimulus_ID) > 0) vc_list$Stimulus_ID[[1]] else NA_real_
prompt_id_var_val <- if (!is.null(vc_list$Prompt_ID) && length(vc_list$Prompt_ID) > 0) vc_list$Prompt_ID[[1]] else NA_real_
participant_id_var_val <- if (!is.null(vc_list$Participant_ID) && length(vc_list$Participant_ID) > 0) vc_list$Participant_ID[[1]] else NA_real_

intercept_est <- res[res$term == "(Intercept)", "Estimate"]
intercept_se <- res[res$term == "(Intercept)", "Std. Error"]

angleleft_est <- res[res$term == "Angleleft", "Estimate"]
angleleft_se <- res[res$term == "Angleleft", "Std. Error"]
angleleft_p_val <- res[res$term == "Angleleft", "Pr(>|z|)"]
angleleft_eff_size <- res[res$term == "Angleleft", "effect_size"]

angleright_est <- res[res$term == "Angleright", "Estimate"]
angleright_se <- res[res$term == "Angleright", "Std. Error"]
angleright_p_val <- res[res$term == "Angleright", "Pr(>|z|)"]
angleright_eff_size <- res[res$term == "Angleright", "effect_size"]

proximity_term_name <- "scale(Proximity, scale = FALSE)"
if (!proximity_term_name %in% res$term) {
    proximity_term_name <- grep("Proximity", res$term, value = TRUE)[1]
}
proximity_est <- res[res$term == proximity_term_name, "Estimate"]
proximity_se <- res[res$term == proximity_term_name, "Std. Error"]
proximity_p_val <- res[res$term == proximity_term_name, "Pr(>|z|)"]
proximity_eff_size <- res[res$term == proximity_term_name, "effect_size"]

n_candidates_term_name <- "scale(n_candidates, scale = FALSE)"
if (!n_candidates_term_name %in% res$term) {
    n_candidates_term_name <- grep("n_candidates", res$term, value = TRUE)[1]
}
n_obj_est <- res[res$term == n_candidates_term_name, "Estimate"]
n_obj_se <- res[res$term == n_candidates_term_name, "Std. Error"]
n_obj_p_val <- res[res$term == n_candidates_term_name, "Pr(>|z|)"]
n_obj_eff_size <- res[res$term == n_candidates_term_name, "effect_size"]

actory_term_name <- "ActorY" # Assuming 'X' is the reference level for Actor factor
actory_est <- res[res$term == actory_term_name, "Estimate"]
actory_se <- res[res$term == actory_term_name, "Std. Error"]
actory_p_val <- res[res$term == actory_term_name, "Pr(>|z|)"]
actory_eff_size <- res[res$term == actory_term_name, "effect_size"]

# Prepare data for data frame
table_data <- list(
    c("StimulusID", "Variance", sprintf("%.7f", stim_id_var_val)),
    c("ParticipantID", "Variance", sprintf("%.7f", participant_id_var_val)),
    c("PromptID", "Variance", sprintf("%.7f", prompt_id_var_val)),
    c("Effect Size Denom", "", sprintf("%.7f", effect_size_denom_t)),

    c("Intercept", "b", sprintf("%.7f", intercept_est)),
    c("", "SE", sprintf("%.7f", intercept_se)),

    c("Angleleft", "b", sprintf("%.7f", angleleft_est)),
    c("", "SE", sprintf("%.7f", angleleft_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleleft_p_val))),
    c("", "Effect size", sprintf("%.7f", angleleft_eff_size)),

    c("Angleright", "b", sprintf("%.7f", angleright_est)),
    c("", "SE", sprintf("%.7f", angleright_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleright_p_val))),
    c("", "Effect size", sprintf("%.7f", angleright_eff_size)),

    c("Proximity", "b", sprintf("%.7f", proximity_est)),
    c("", "SE", sprintf("%.7f", proximity_se)),
    c("", "p", sprintf("%.7f", as.numeric(proximity_p_val))),
    c("", "Effect size", sprintf("%.7f", proximity_eff_size)),

    c("#Object", "b", sprintf("%.7f", n_obj_est)),
    c("", "SE", sprintf("%.7f", n_obj_se)),
    c("", "p", sprintf("%.7f", as.numeric(n_obj_p_val))),
    c("", "Effect size", sprintf("%.7f", n_obj_eff_size)),

    c("ActorY", "b", sprintf("%.7f", actory_est)),
    c("", "SE", sprintf("%.7f", actory_se)),
    c("", "p", sprintf("%.7f", as.numeric(actory_p_val))),
    c("", "Effect size", sprintf("%.7f", actory_eff_size))
)

# Convert to data frame
df_output <- do.call(rbind, lapply(table_data, function(x) data.frame(V1=x[1], V2=x[2], V3=x[3], stringsAsFactors=FALSE)))

# Print only the third column for easy copy-pasting to Excel
write.table(df_output[,3, drop=FALSE], file="", sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

2.7443130
0.4037442
NA
2.5373067
7.4138498
0.5270114
-3.2358288
0.4217099
0.0000000
-1.2753006
-3.2180958
0.4238400
0.0000000
-1.2683117
-1.3047983
0.1677296
0.0000000
-0.5142454
-0.6700431
0.1518226
0.0000102
-0.2640765
-1.0032615
0.2383074
0.0000255
-0.3954041
